In [4]:
import gym
import numpy as np

In [5]:
env = gym.make('FrozenLake-v1')
MAX_STEP_PER_EPISODE = 1000

In [6]:
def target_policy(q_function,state):
    action = np.zeros_like(q_function[state])
    optimal = np.argmax(q_function[state])
    action[optimal] = 1.0
    return action

In [7]:
def behavior_policy(env, state):
    return np.ones(env.nA) / env.nA

In [26]:
def off_policy_mc(env = env, 
                  behavior_policy=behavior_policy, 
                  target_policy = target_policy, 
                  episodes = 50000, 
                  discount = 1.0):
    """
    Apply off-policy Monte Carlo method with importance sampling to find optimal policy
    
    Args:
        env: environment
        behavior_policy: a function takes q and state, return the soft probs of actions
        target_policy: a function takes env and state, return the greedy probs of actions
        episodes: number of episodes
        discount: discount factor
        
    Return:
        Q: Action-State function
        
    """
    #RL: an intro page 110 and 111
    Q = np.zeros((env.nS, env.nA))
    C = np.zeros((env.nS, env.nA))
    
    for epi in range(1, episodes + 1):
        episode = []
        state = env.reset()
        for t in range(MAX_STEP_PER_EPISODE):
            prob = behavior_policy(env, state)
            action = np.random.choice(np.arange(env.nA), p=prob)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state
        
        
        G = 0
        W = 1.0
        
        for t in range(len(episode) - 1, -1, -1):
            state, action, reward = episode[t];
            G = discount * G + reward
            C[state][action] += W
            Q[state][action] += W / C[state][action] * (G - Q[state][action])
            
            # if the action taken would not be taken by target policy
            # the prob is zero, therefore break here
            if action != np.argmax(target_policy(Q, state)):
                break
            
            W *= 1.0 / behavior_policy(env, state)[action]
    return Q
    

In [27]:
q = off_policy_mc()

In [28]:
env.reset()
env.render()
env.close()


SFFF
FHFH
FFFH
HFFG


In [29]:
value = np.zeros(env.nS)
for i in range(env.nS):
    value[i] = np.max(q[i])
print(value.reshape(4,4))

[[0.99999967 0.99999917 0.99999695 0.        ]
 [0.9994627  0.         0.52557485 0.        ]
 [0.99929385 0.99539226 0.59722222 0.        ]
 [0.         0.96106195 0.95675676 0.        ]]
